<a href="https://colab.research.google.com/github/snigdha2606/Automatic-Video-Summarization-and-Hashtag-Generation/blob/main/hashtag_generation_projectcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency r

In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 11.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
import av
import numpy as np
from transformers import (
    AutoImageProcessor,
    AutoTokenizer,
    VisionEncoderDecoderModel,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# Step 1: Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 2: Load pretrained processor, tokenizer, and video captioning model
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
caption_tokenizer = AutoTokenizer.from_pretrained("gpt2")
caption_model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

# Step 5: Load the text generation model for hashtag generation
model_name = "NousResearch/Llama-2-7b-chat-hf"
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = {"": 0}
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Step 6: Load the LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import os

def extract_frames(video_path, frame_rate):
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while True:
        success, image = vidcap.read()
        if not success:
            break
        if count % frame_rate == 0:
            frames.append(image)
        count += 1
    vidcap.release()
    return np.array(frames)

def extract_motion(frames):
    motion_features = []
    prev_frame = frames[0]
    for frame in frames[1:]:
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        motion_feature = np.mean(magnitude)
        motion_features.append(motion_feature)
        prev_frame = frame
    return np.array(motion_features)

def apply_kmeans_motion(frames, motion_features, k):
    if len(motion_features) < k:
        k = len(motion_features)  # Adjust the number of clusters
    features = np.array(motion_features).reshape(-1, 1)
    kmeans = KMeans(n_clusters=k, random_state=42).fit(features)
    labels = kmeans.labels_
    key_frames = []
    for i in range(k):
        index = np.where(labels == i)[0]
        if len(index) > 0:
            key_frames.append(frames[index[0]])
    return np.array(key_frames)

def extract_histograms(frames):
    histograms = []
    for frame in frames:
        hist = cv2.calcHist([frame], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        histograms.append(hist)
    return histograms

def apply_kmeans_color(histograms, k):
    if len(histograms) < k:
        k = len(histograms)  # Adjust the number of clusters
    samples = np.array(histograms)
    samples = samples.reshape(-1, samples.shape[-1])
    kmeans = KMeans(n_clusters=k, random_state=42).fit(samples)
    labels = kmeans.labels_
    key_frame_indices = []
    for i in range(k):
        index = np.where(labels == i)[0]
        if len(index) > 0:
            key_frame_indices.append(index[0])
    return np.array(key_frame_indices)

def detect_events(motion_features, threshold):
    events = []
    event_start = 0
    event_end = 0
    for i in range(len(motion_features)):
        if motion_features[i] > threshold:
            if event_start == 0:
                event_start = i
            event_end = i
        else:
            if event_start != 0:
                events.append((event_start, event_end))
                event_start = 0
                event_end = 0
    if event_start != 0:
        events.append((event_start, event_end))
    return events

def extract_event_frames(frames, events):
    event_frames = []
    for event in events:
        start_frame = event[0]
        end_frame = event[1]
        for i in range(start_frame, end_frame+1):
            event_frames.append(frames[i])
    return np.array(event_frames)

def create_event_summary_video(event_frames, output_video_path, frame_rate=30):
    if len(event_frames) == 0:
        print("No event frames to save.")
        return

    height, width, layers = event_frames[0].shape
    size = (width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use the 'mp4v' codec
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, size)

    for frame in event_frames:
        out.write(frame)

    out.release()
    print(f"Video summarization completed. Event frames video saved in {output_video_path}")

def summarize_video(video_path, frame_rate, k_motion, k_color, threshold, max_frames, output_video_path, output_frame_rate):
    frames = extract_frames(video_path, frame_rate)[:max_frames]
    motion_features = extract_motion(frames)
    key_frames_motion = apply_kmeans_motion(frames, motion_features, k_motion)
    histograms = extract_histograms(frames)
    key_frame_indices_color = apply_kmeans_color(histograms, k_color)
    key_frames_color = frames[key_frame_indices_color]
    events = detect_events(motion_features, threshold)
    event_frames = extract_event_frames(frames, events)
    create_event_summary_video(event_frames, output_video_path, frame_rate=output_frame_rate)

    print(f"Video summarization completed. Key frames based on motion: {len(key_frames_motion)}. Key frames based on color: {len(key_frames_color)}. Event frames: {len(event_frames)}.")

if __name__ == '__main__':
    video_path = '/content/fire.mp4'
    frame_rate = 10  # Extract one frame every n seconds
    k_motion = 10  # Number of motion-based key frames
    k_color = 10  # Number of color-based key frames
    threshold = 3  # Motion threshold to detect events
    max_frames = 1000  # Maximum number of frames to process
    output_video_path = '/content/fire_summary.mp4'
    output_frame_rate = 6  # Frame rate of the output video

    summarize_video(video_path, frame_rate, k_motion, k_color, threshold, max_frames, output_video_path, output_frame_rate)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Video summarization completed. Event frames video saved in /content/fire_summary.mp4
Video summarization completed. Key frames based on motion: 10. Key frames based on color: 10. Event frames: 28.


In [ ]:
# Step 3: Load video and extract evenly spaced frames
video_path = "/content/fire_summary.mp4"  # Change this to your video path
container = av.open(video_path)
seg_len = container.streams.video[0].frames
clip_len = caption_model.config.encoder.num_frames
indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))
frames = []
container.seek(0)
for i, frame in enumerate(container.decode(video=0)):
    if i in indices:
        frames.append(frame.to_ndarray(format="rgb24"))

# Step 4: Generate caption for the video frames
gen_kwargs = {
    "min_length": 2,
    "max_length": 20,
    "num_beams": 12,
}
pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
tokens = caption_model.generate(pixel_values, **gen_kwargs)
caption = caption_tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
print("Generated Caption:", caption)


logging.set_verbosity(logging.CRITICAL)

# Step 7: Generate hashtags from the caption
prompt = f"Create hashtags of the following text: {caption}"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
generated_text = result[0]['generated_text']
print("Generated Text for Hashtags:", generated_text)

# Step 8: Extract hashtags using regular expressions
import re
hashtags = re.findall(r"#\w+", generated_text)
print("Generated Hashtags:", hashtags)

Generated Caption: A group of people are outside in the middle of a fire and are trying to extinguish it
Generated Text for Hashtags: <s>[INST] Create hashtags of the following text: A group of people are outside in the middle of a fire and are trying to extinguish it [/INST]  Here are some possible hashtags for the text you provided:
 nobody #firefighters #safetyfirst #teamwork #emergencyresponse #fireextinguisher #flames #rescue #heroes #fightingfire
Generated Hashtags: ['#firefighters', '#safetyfirst', '#teamwork', '#emergencyresponse', '#fireextinguisher', '#flames', '#rescue', '#heroes', '#fightingfire']
